In [1]:
# import libraries
import os
import requests

import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.utils import pad_sequences



print(tf.__version__)

  Using cached tensorflow_datasets-4.9.7-py3-none-any.whl.metadata (9.6 kB)
  Using cached immutabledict-4.2.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached promise-2.3.tar.gz (19 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached simple_parsing-0.1.6-py3-none-any.whl.metadata (7.3 kB)
  Using cached tensorflow_metadata-1.16.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached etils-1.11.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
  Using cached importlib_resources-6.4.5-py3-none-any.whl.metadata (4.0 kB)
  Using cached docstring_parser-0.16-py3-none-any.whl.metadata (3.0 kB)
Using cached tensorflow_datasets-4.9.7-py3-none-any.whl (5.3 MB)
Using cached etils-1.11.0-py3-none-any.whl (165 kB)
Using cached immutabledict-4.2.1-py3-none-any.whl (4.7 kB)
   ---------------------------------------- 0.0/25.1 MB ? eta -:--:--
   ------------------------------------

In [9]:
# get data files
import os
import requests

# Define file paths
train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

# URLs for the datasets
train_url = "https://cdn.freecodecamp.org/project-data/sms/train-data.tsv"
test_url = "https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv"

# Function to download a file
def download_file(url, file_path):
    response = requests.get(url)
    if response.status_code == 200:
        with open(file_path, "wb") as file:
            file.write(response.content)
        print(f"Downloaded {file_path}")
    else:
        print(f"Failed to download {file_path}. Status code: {response.status_code}")

# Download the datasets
if not os.path.exists(train_file_path):
    download_file(train_url, train_file_path)

if not os.path.exists(test_file_path):
    download_file(test_url, test_file_path)


In [10]:
train_dataset = pd.read_csv('train-data.tsv',sep='\t',header=None,names=['class', 'text'],encoding='utf-8')
valid_dataset = pd.read_csv('valid-data.tsv',sep='\t',header=None,names=['class', 'text'],encoding='utf-8')

In [11]:
VOCAB_SIZE = 88584
BATCH_SIZE = 64

In [12]:
# Preprocessing
def preprocess(df):
    data = df.copy()
    data.loc[data['class'] == 'ham','class'] = 0
    data.loc[data['class'] == 'spam','class'] = 1
    labels = data.pop('class').astype(np.float32)
    new_data = []
    for i in data['text']:
      new_data.append(i)
    return new_data, labels

train_data, train_labels = preprocess(train_dataset)
valid_data, valid_labels = preprocess(valid_dataset)

# Calculating the maximum message length
MAXLEN_train = max(train_data, key=len)
MAXLEN_valid = max(valid_data, key=len)
MAXLEN = max(len(MAXLEN_train.strip()), len(MAXLEN_valid.strip()))

In [13]:
# Taking words indexes from preproc lib
word_index = imdb.get_word_index()

# Function for encoding text message
def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return pad_sequences([tokens], MAXLEN)[0]

# Func for prep text message for model
def prep_to_mod(data_for_prep):
    new_data = []
    for i in data_for_prep:
        new_data.append(encode_text(i))
    return np.array(new_data)

# Prep data for model
new_train_data = prep_to_mod(train_data)
new_valid_data = prep_to_mod(valid_data)

In [14]:
# Creating model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

In [22]:
# Fitting model
model.fit(new_train_data, train_labels, epochs=5, validation_split = 0.2)

Epoch 1/5
105/105 ━━━━━━━━━━━━━━━━━━━━ 18s 158ms/step - acc: 0.8483 - loss: 0.4183 - val_acc: 0.9701 - val_loss: 0.1255
Epoch 2/5
105/105 ━━━━━━━━━━━━━━━━━━━━ 17s 164ms/step - acc: 0.9650 - loss: 0.1357 - val_acc: 0.9797 - val_loss: 0.0812
Epoch 3/5
105/105 ━━━━━━━━━━━━━━━━━━━━ 16s 156ms/step - acc: 0.9722 - loss: 0.0967 - val_acc: 0.9785 - val_loss: 0.0659
Epoch 4/5
105/105 ━━━━━━━━━━━━━━━━━━━━ 17s 161ms/step - acc: 0.9850 - loss: 0.0567 - val_acc: 0.9821 - val_loss: 0.0620
Epoch 5/5
105/105 ━━━━━━━━━━━━━━━━━━━━ 16s 150ms/step - acc: 0.9881 - loss: 0.0427 - val_acc: 0.9856 - val_loss: 0.0510


In [23]:
results = model.evaluate(new_valid_data, valid_labels)
print(results)

44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - acc: 0.9811 - loss: 0.0542
[0.05789817497134209, 0.9820402264595032]


In [24]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    encoded_text = encode_text(pred_text)
    pred = np.zeros((1, MAXLEN))
    pred[0] = encoded_text
    result = model.predict(pred).flatten()[0]
    label = ""
    if result > 0.5:
        label = "spam"
    else:
        label = "ham"
    prediction = [result, label]
    return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
[0.0011350012, 'ham']


In [25]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
You passed the challenge. Great job!
